In [1]:
import cv2
import tensorflow as tf
import numpy as np
import os
import sys
import math


from config_tf import config
from config_tf import Path

class DataLoader(tf.keras.utils.Sequence):
    def __init__(self, x_set, y_set, batch_size, shuffle = True):
        super(DataLoader, self).__init__()
        # Here, `x_set` is list of path to the images
        # and `y_set` are the associated classes.
        self.x = x_set
        self.y = y_set
        self.batch_size = batch_size
        self.clip_len = 16
        self.crop_size = 112
        
        # Prepare a mapping between the label names (strings) and indices (ints)
        self.label2index = {label: index for index, label in enumerate(sorted(set(self.y)))}
        # Convert the list of label names into an array of label indices
        self.label_array = np.array([self.label2index[label] for label in self.y], dtype=int)

        self.shuffle = shuffle
        self.on_epoch_end()
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.x))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)
    
    def __getitem__(self, index):
        indexes =  self.indexes[index * self.batch_size : (index + 1) * self.batch_size]
        clip_dir_list = [self.x[k] for k in indexes]
        label_list = [self.y[k] for k in indexes]
        X = self.__get_buffer(clip_dir_list)
        return X, label_list
            

    def __get_buffer(self, clip_dir_list):
        batch = np.empty((self.batch_size, self.clip_len, self.crop_size, self.crop_size, 3), np.dtype('float32'))
        for i, clip_dir in enumerate(clip_dir_list):
            buffer = self.__get_frames(clip_dir)
            buffer = self.__crop(buffer)
            buffer = self.__normalize(buffer)

            ## TODO: add rand flip
            batch[i]=buffer
        return batch

    
    def __get_frames(self, clip_dir):
        frames = sorted([os.path.join(clip_dir, img) for img in os.listdir(clip_dir)],key=lambda x:int(x.split('/')[-1][:-4]))
        frames = self.__timing_jitter(frames)
        frame_count = len(frames)
        frame_height, frame_width = cv2.imread(frames[0]).shape[0:2]
        buffer = np.empty((frame_count, frame_height, frame_width, 3), np.dtype('float32'))
        for i, frame_name in enumerate(frames):
            frame = np.array(cv2.imread(frame_name)).astype(np.float64)
            buffer[i] = frame
        return buffer

    def __timing_jitter(self, frames):
        # randomly select time index for temporal jittering
        time_index = np.random.randint(len(frames) - self.clip_len)
        frames = frames[time_index:time_index +self.clip_len]
        return frames
        

    
    def __crop(self, buffer):
        height_index = np.random.randint(buffer.shape[1] - self.crop_size)
        width_index = np.random.randint(buffer.shape[2] - self.crop_size)
        
        buffer = buffer[:, height_index:height_index + self.crop_size,
                        width_index:width_index + self.crop_size, :]

        return buffer
    
    def __normalize(self, buffer):
        for i, frame in enumerate(buffer):
            frame -= np.array([[[90.0, 98.0, 102.0]]])
            #added nomalize data to [0,1]
            frame = frame / 255.0
            buffer[i] = frame

        return buffer

    def __random_flip(self, buffer):
        pass

def load_data(dataset):
    labels = []
    fnames = []
    _, frames_dir = Path.db_dir(dataset)
    train_folder = os.path.join(frames_dir, "train")
    for label in sorted(os.listdir(train_folder)):
        for fname in os.listdir(os.path.join(train_folder, label)):
            fnames.append(os.path.join(train_folder, label, fname))
            labels.append(label)

    return fnames, labels
        


2021-07-23 17:00:20.068460: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [ ]:
x_train, y_train = load_data(config.dataset)
loader = DataLoader(x_train, y_train, config.batch_size)

for i, (clips, label) in enumerate(loader):
    if(i<10):
        first_clip = clips[0,0,:,:,:]
        cv2.imwrite("test_" + str(i+1) + "_" + label[0] + ".jpg",first_clip)